In [1]:
import torch
print(f"torch version {torch.__version__}")
from torch import nn
import datetime

# Importing TensorBoard libraries for visuvalization
from torch.utils.tensorboard import SummaryWriter

# Importing Ray Tune libraries for hyperparameter tuning
from ray import train, tune

# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device
print(f"Device = {device}")

torch version 1.13.1
Device = cpu


In [2]:
X = torch.tensor([[1.], [2.], [3.], [4.], [5.], [6.], [7.], [8.], [9.], [10.]])
y = torch.tensor([[2.], [4.], [6.], [8.], [10.], [12.], [14.], [16.], [18.], [20.]])
#X_test = torch.tensor([[2.5], [22.], [-6.], [.25],])
#y_test = torch.tensor([[5.], [44.], [-12.], [.5],])
X_test = torch.tensor([[40.]])
y_test = torch.tensor([[80.]])
print(X.size())
print(y.size())
print(X_test.size())
print(y_test.size())

tboard_dir = "runs/pytoch_learning/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S");
print(tboard_dir)
writer = SummaryWriter(tboard_dir)

torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([1, 1])
torch.Size([1, 1])
runs/pytoch_learning/20231024-174408


In [3]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, X, y, transform=None, target_transform=None):
        self.X = X
        self.y = y
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):

        return self.X[idx], self.y[idx]

In [4]:
full_dataset = MyDataset(X,y)
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [8, 2])

In [11]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=True)

In [8]:
train_features, train_labels = next(iter(train_dataloader))

tensor([[4.],
        [9.]])
tensor([[ 8.],
        [18.]])


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_stack = nn.Linear(1, 1)
        
    def forward(self, x):
        logits = self.linear_stack(x)
        return logits
    
model = NeuralNetwork()
print(f"Model structure : {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

In [ ]:
#learning_rate = 1e-3
learning_rate = 0.001
batch_size = 64
epochs = 5

# Initialize the loss function
loss_fn = nn.MSELoss()
print(f"Loss function : {loss_fn} \n")

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
print(f"Optimizer : {optimizer}")

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")



def train_loop(X, y, model, loss_fn, optimizer):

    #print("****** Training Start *******\n")
    
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()

    pred = model(X)
    #print(f"Prediction = {pred} \n")
    
    loss = loss_fn(pred, y)
    #print(f"Training Loss = {loss}")

    # Backpropagation
    
    # parameters before bp
    #print("**** parms before bp ****")
    #for name, param in model.named_parameters():
    #    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]}")
    #print("\n")
    
    loss.backward()
    
    # parameters after bp, before optimizer
    #print("**** parms after bp, before optimizer ****")
    #for name, param in model.named_parameters():
    #    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]}")
    #print("\n")
    
    # Optimization
    
    optimizer.step()
    
    # parameters after optimization
    #print("**** parms after optimizer ****")
    #for name, param in model.named_parameters():
    #    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]}")
    #print("\n")
    
    optimizer.zero_grad()
    
    #print("****** Training End *******\n")
    
    return loss

In [ ]:
def test_loop(X_test, y_test, model, loss_fn):
    
    #print("****** Testing Start ******\n")
    
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        pred_test = model(X_test)
        #print(f"Test Prediction = {pred_test}")
        test_loss = loss_fn(pred_test, y_test)
        #print(f"Test Loss = {test_loss}")
    
    #print("****** Testing End ******\n")
    
    return test_loss

In [ ]:
parms = []
weight = torch.empty((1,1))
bias = torch.empty((1,1))

for t in range (100):
    
    train_loss = train_loop(train_dataloader, loss_fn, optimizer)
    
    for name, param in model.named_parameters():
    #    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]}")
        parms.append(param[:2])
    
    test_loss = test_loop(test_dataloader, model, loss_fn)

    weight = parms[0]
    bias = parms[1]
    
    print(f"Epoch {t+1}, Weight = {weight.detach().numpy()[0][0]}, Bias = {bias.detach().numpy()[0]}, Training Loss = {train_loss}, Testing Loss = {test_loss}")
    
    writer.add_scalars('Training vs. Test Loss',
                            {'Training' : train_loss, 'Test' : test_loss,}, t)
    
    writer.add_scalar('Training Loss', train_loss, t)
    
    writer.flush()

print("\n***** End of processing *****")